In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
pymysql.install_as_MySQLdb()

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [4]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
143047,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
143048,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"
143049,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller
143050,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History"


In [5]:
basics['genres_split'] = basics['genres'].str.split(',')
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,[Drama]
3,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War","[Drama, War]"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
...,...,...,...,...,...,...,...,...,...,...
143047,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama,[Drama]
143048,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"
143049,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller,[Thriller]
143050,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History","[Drama, History]"


In [13]:
genres_split = basics['genres'].str.split(",")

# explode the series and take the unique entries only
unique_genres = genres_split.explode().unique()
unique_genres

array(['Comedy', 'Fantasy', 'Romance', 'Drama', 'War', 'Horror', 'Sci-Fi',
       'Mystery', 'Adventure', 'Musical', 'Action', 'Crime', 'Thriller',
       'Music', 'Animation', 'Family', 'History', 'Biography', 'Sport',
       'Western', 'Adult', 'Short', 'Reality-TV', 'News', 'Talk-Show',
       'Game-Show'], dtype=object)

In [15]:
exploded_genres = basics.explode('genres_split')
exploded_genres

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...
143048,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller",Thriller
143049,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020.0,NaN,84,Thriller,Thriller
143050,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History",Drama
143050,tt9916362,movie,Coven,Akelarre,0,2020.0,NaN,92,"Drama,History",History


In [16]:
unique_genres = sorted(exploded_genres['genres_split'].unique())

In [18]:
# save just the tconst and genres as a new df
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama


In [19]:
# make a dictionary with list of unique genres as key and new integer id as values
genre_id_map = dict(zip(unique_genres, range(len(unique_genres))))
genre_id_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

In [20]:
genre_lookup = pd.DataFrame({'Genre_Name': genre_id_map.keys(),
                            'Genre_ID': genre_id_map.values()})

genre_lookup.head()

,Genre_Name,Genre_ID
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4
